In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np
import random

Batch_Size = 250
num_epochs = 200
learning_rate = 1e-5
samples = 30000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset


for i in range(samples):
    
    switch = random.randint(0,1)
    if switch==0:
        img = Image.new('RGB', (32, 32), color = 'white')
        row,col,ch= np.shape(img)
        mean = random.randint(1,6)
        var = random.random()
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2_pos1))

        img_1 = Image.new('RGB',  (32, 32), color = 'white')
        row_1,col_1,ch_1= np.shape(img_1)
        mean_1 = random.randint(1,6)
        var_1 = random.random()
        sigma_1 = var_1**0.5
        gauss_pos2 = np.random.normal(mean_1,sigma_1,(row_1,col_1,ch_1))
        noisy_pos2 = img_1 + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw_1 = ImageDraw.Draw(im2_pos2)
        draw_1.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data2.append(np.asarray(im2_pos2))
    elif switch==1:
        img_blank = Image.new('RGB',  (32, 32), color = 'white')
        row_blank,col_blank,ch_blank= np.shape(img_blank)
        mean_blank = random.randint(1,6)
        var_blank = random.random()
        sigma_blank = var_blank**0.5
        gauss_blank_pos1 = np.random.normal(mean_blank,sigma_blank,(row_blank,col_blank,ch_blank))
        noisy_blank_pos1 = img_blank + gauss_blank_pos1
        im2_blank_pos1 = Image.fromarray(noisy_blank_pos1,'RGB')
        data1.append(np.asarray(im2_blank_pos1))

        img_blank_1 = Image.new('RGB', (32, 32), color = 'white')
        row_blank_1,col_blank_1,ch_blank_1= np.shape(img_blank_1)
        mean_blank_1 = random.randint(1,6)
        var_blank_1 = random.random()
        sigma_blank_1 = var_blank_1**0.5
        gauss_blank_pos2 = np.random.normal(mean_blank_1,sigma_blank_1,(row_blank_1,col_blank_1,ch_blank_1))
        noisy_blank_pos2 = img_blank_1 + gauss_blank_pos2
        im2_blank_pos2 = Image.fromarray(noisy_blank_pos2,'RGB')
        data2.append(np.asarray(im2_blank_pos2))



#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB',  (32, 32), color = 'white')
row,col,ch= np.shape(img)
mean = random.randint(1,6)
var = random.random()
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b=b[:,:,0]
b = torch.from_numpy(b)
print(np.shape(b))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = y0[:,:,:,0]
y1 = y1[:,:,:,0]

print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=64
        )
        self.encoder_output_layer_1 = nn.Linear(
            in_features=64, out_features=32
        )
        self.encoder_output_layer_2 = nn.Linear(
            in_features=32, out_features=16
        )
        self.encoder_output_layer_3 = nn.Linear(
            in_features=16, out_features=8
        )
        self.encoder_output_layer_4 = nn.Linear(
            in_features=8, out_features=1
        )




    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        code = self.encoder_output_layer_1(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_2(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_3(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_4(code)
        code = torch.sigmoid(code)
        return code

#  use gpu if available

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=1024)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-4)



for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        x = x.view(-1, 1024)
        y = y.view(-1, 1024)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    b = b.view(-1,1024)
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print(np.shape(x[0]))
    print("x: ", model(x[0]))
    print("y:", model(y[0]))
    print("b:", model(b))



torch.Size([32, 32])
(30000, 32, 32)
epoch [1/200], loss:-0.0051
torch.Size([1024])
x:  tensor([0.9056], grad_fn=<SigmoidBackward>)
y: tensor([0.8440], grad_fn=<SigmoidBackward>)
b: tensor([[0.8436]], grad_fn=<SigmoidBackward>)
epoch [2/200], loss:-0.0390
torch.Size([1024])
x:  tensor([0.5931], grad_fn=<SigmoidBackward>)
y: tensor([0.7224], grad_fn=<SigmoidBackward>)
b: tensor([[0.7138]], grad_fn=<SigmoidBackward>)
epoch [3/200], loss:-0.1143
torch.Size([1024])
x:  tensor([0.3157], grad_fn=<SigmoidBackward>)
y: tensor([0.3381], grad_fn=<SigmoidBackward>)
b: tensor([[0.5259]], grad_fn=<SigmoidBackward>)
epoch [4/200], loss:-0.2852
torch.Size([1024])
x:  tensor([0.9799], grad_fn=<SigmoidBackward>)
y: tensor([0.9783], grad_fn=<SigmoidBackward>)
b: tensor([[0.1546]], grad_fn=<SigmoidBackward>)
epoch [5/200], loss:-0.3615
torch.Size([1024])
x:  tensor([0.9740], grad_fn=<SigmoidBackward>)
y: tensor([0.9868], grad_fn=<SigmoidBackward>)
b: tensor([[0.0514]], grad_fn=<SigmoidBackward>)
epoch [6

epoch [44/200], loss:-0.4369
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[2.5252e-05]], grad_fn=<SigmoidBackward>)
epoch [45/200], loss:-0.4355
torch.Size([1024])
x:  tensor([0.9999], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[2.2247e-05]], grad_fn=<SigmoidBackward>)
epoch [46/200], loss:-0.4333
torch.Size([1024])
x:  tensor([4.8247e-05], grad_fn=<SigmoidBackward>)
y: tensor([1.6148e-05], grad_fn=<SigmoidBackward>)
b: tensor([[2.1060e-05]], grad_fn=<SigmoidBackward>)
epoch [47/200], loss:-0.4373
torch.Size([1024])
x:  tensor([0.9999], grad_fn=<SigmoidBackward>)
y: tensor([1.], grad_fn=<SigmoidBackward>)
b: tensor([[1.9631e-05]], grad_fn=<SigmoidBackward>)
epoch [48/200], loss:-0.4339
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.], grad_fn=<SigmoidBackward>)
b: tensor([[1.8472e-05]], grad_fn=<SigmoidBackward>)
epoch [49/200], loss

epoch [86/200], loss:-0.4371
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.], grad_fn=<SigmoidBackward>)
b: tensor([[2.3382e-06]], grad_fn=<SigmoidBackward>)
epoch [87/200], loss:-0.4366
torch.Size([1024])
x:  tensor([3.0079e-06], grad_fn=<SigmoidBackward>)
y: tensor([4.1138e-06], grad_fn=<SigmoidBackward>)
b: tensor([[2.2495e-06]], grad_fn=<SigmoidBackward>)
epoch [88/200], loss:-0.4369
torch.Size([1024])
x:  tensor([4.6091e-06], grad_fn=<SigmoidBackward>)
y: tensor([2.6370e-06], grad_fn=<SigmoidBackward>)
b: tensor([[2.2593e-06]], grad_fn=<SigmoidBackward>)
epoch [89/200], loss:-0.4371
torch.Size([1024])
x:  tensor([1.7226e-06], grad_fn=<SigmoidBackward>)
y: tensor([7.1685e-06], grad_fn=<SigmoidBackward>)
b: tensor([[2.1883e-06]], grad_fn=<SigmoidBackward>)
epoch [90/200], loss:-0.4351
torch.Size([1024])
x:  tensor([2.8129e-06], grad_fn=<SigmoidBackward>)
y: tensor([4.8391e-06], grad_fn=<SigmoidBackward>)
b: tensor([[2.1297e-06]], grad_fn=<SigmoidBac

epoch [128/200], loss:-0.4360
torch.Size([1024])
x:  tensor([1.3535e-06], grad_fn=<SigmoidBackward>)
y: tensor([2.4050e-06], grad_fn=<SigmoidBackward>)
b: tensor([[1.7676e-06]], grad_fn=<SigmoidBackward>)
epoch [129/200], loss:-0.4360
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.], grad_fn=<SigmoidBackward>)
b: tensor([[1.7462e-06]], grad_fn=<SigmoidBackward>)
epoch [130/200], loss:-0.4351
torch.Size([1024])
x:  tensor([2.6015e-06], grad_fn=<SigmoidBackward>)
y: tensor([3.1389e-06], grad_fn=<SigmoidBackward>)
b: tensor([[1.7452e-06]], grad_fn=<SigmoidBackward>)
epoch [131/200], loss:-0.4346
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.], grad_fn=<SigmoidBackward>)
b: tensor([[1.7172e-06]], grad_fn=<SigmoidBackward>)
epoch [132/200], loss:-0.4375
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.], grad_fn=<SigmoidBackward>)
b: tensor([[1.7724e-06]], grad_fn=<SigmoidBackward>)
epoch [133/

epoch [170/200], loss:-0.4375
torch.Size([1024])
x:  tensor([1.4994e-06], grad_fn=<SigmoidBackward>)
y: tensor([2.4241e-06], grad_fn=<SigmoidBackward>)
b: tensor([[1.6520e-06]], grad_fn=<SigmoidBackward>)
epoch [171/200], loss:-0.4369
torch.Size([1024])
x:  tensor([2.7624e-06], grad_fn=<SigmoidBackward>)
y: tensor([1.9793e-06], grad_fn=<SigmoidBackward>)
b: tensor([[1.6131e-06]], grad_fn=<SigmoidBackward>)
epoch [172/200], loss:-0.4374
torch.Size([1024])
x:  tensor([3.2973e-06], grad_fn=<SigmoidBackward>)
y: tensor([3.5811e-06], grad_fn=<SigmoidBackward>)
b: tensor([[1.7555e-06]], grad_fn=<SigmoidBackward>)
epoch [173/200], loss:-0.4371
torch.Size([1024])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.5339e-06]], grad_fn=<SigmoidBackward>)
epoch [174/200], loss:-0.4360
torch.Size([1024])
x:  tensor([2.2879e-06], grad_fn=<SigmoidBackward>)
y: tensor([3.6781e-05], grad_fn=<SigmoidBackward>)
b: tensor([[1.6687e-06]], grad_fn=<S

In [351]:
data3=[]

for i in range(5):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = random.randint(1,6)
    var = random.random()
    sigma = var**0.5
    gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
    noisy_pos1 = img + gauss_pos1
    im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
    draw = ImageDraw.Draw(im2_pos1)
    draw.rectangle([9,12,12,15],fill='red',outline='red')
    data3.append(np.asarray(im2_pos1))

    y3 = np.array(data3, dtype=np.float32)
    #y1 = np.array(data2, dtype=np.float32)
    y3 = y3[:,:,:,0]
    #y1 = y1[:,:,:,0]
    x_1 = torch.from_numpy(y3)
    print("x: ", model(x_1[i].view(-1,784)))

x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
